In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def bert_module(query, key, value, embed_dim, num_head, i):
    
    # Multi headed self-attention
    attention_output = layers.MultiHeadAttention(
        num_heads=num_head,
        key_dim=embed_dim // num_head,
        name="encoder_{}/multiheadattention".format(i)
    )(query, key, value, use_causal_mask=True)
    
    # Add & Normalize
    attention_output = layers.Add()([query, attention_output])  # Skip Connection
    attention_output = layers.LayerNormalization(epsilon=1e-6)(attention_output)
    
    # Feedforward network
    ff_net = keras.models.Sequential([
        layers.Dense(2 * embed_dim, activation='relu', name="encoder_{}/ffn_dense_1".format(i)),
        layers.Dense(embed_dim, name="encoder_{}/ffn_dense_2".format(i)),
    ])

    # Apply Feedforward network
    ffn_output = ff_net(attention_output)

    # Add & Normalize
    ffn_output = layers.Add()([attention_output, ffn_output])  # Skip Connection
    ffn_output = layers.LayerNormalization(epsilon=1e-6)(ffn_output)
    
    return ffn_output

In [5]:
def get_sinusoidal_embeddings(sequence_length, embedding_dim):
    position_enc = np.array([
        [pos / np.power(10000, 2. * i / embedding_dim) for i in range(embedding_dim)]
        if pos != 0 else np.zeros(embedding_dim)
        for pos in range(sequence_length)
    ])
    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2])  # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2])  # dim 2i+1
    return tf.cast(position_enc, dtype=tf.float32)

In [6]:
N = 20 # vocab_size

vocabs = ['word_' + str(i) for i in range(N)]

vocab_map = {}
for i in range(len(vocabs)):
    vocab_map[vocabs[i]] = i

In [7]:
def get_accuracy_prob(embed_dim):
    
    pairs = []

    for i in vocabs:
        for j in vocabs:
            for k in vocabs:
                if i != j and i != k and j != k:
                    pairs.append((i,j,k))

    indicator = np.random.choice([0, 1], size=len(pairs), p=[0.5, 0.5])

    pairs_train = [pairs[i] for i in range(len(indicator)) if indicator[i] == 1]
    pairs_test = [pairs[i] for i in range(len(indicator)) if indicator[i] == 0]
    
    sentences_train = []
    sentences_number_train = []
    sentences_test_a = []
    sentences_number_test_a = []
    sentences_test_b = []
    sentences_number_test_b = []

    x_masked_train = []
    y_masked_labels_train = []
    x_masked_test_a = []
    y_masked_labels_test_a = []
    x_masked_test_b = []
    y_masked_labels_test_b = []

    for _ in range(25000):

        [(a,b,c), (d,e,f)] = random.sample(pairs_train, 2)

        temp = [a, b, c, a, d, e, f, d]
        sentences_train.append(temp)
        sentences_number_train.append([vocab_map[i] for i in temp])
        x_masked_train.append([vocab_map[i] for i in temp])
        y_masked_labels_train.append([vocab_map[i] for i in temp][1:])

        [(a,b,c), (d,e,f)] = random.sample(pairs_train, 2)

        temp = [a, b, c, b, d, e, f, e]
        sentences_train.append(temp)
        sentences_number_train.append([vocab_map[i] for i in temp])
        x_masked_train.append([vocab_map[i] for i in temp])
        y_masked_labels_train.append([vocab_map[i] for i in temp][1:])



    for _ in range(25000):

        [(a,b,c), (d,e,f)] = random.sample(pairs_test, 2)

        temp = [a, b, c, a, d, e, f, d]
        sentences_test_a.append(temp)
        sentences_number_test_a.append([vocab_map[i] for i in temp])
        x_masked_test_a.append([vocab_map[i] for i in temp])
        y_masked_labels_test_a.append([vocab_map[i] for i in temp][1:])

        [(a,b,c), (d,e,f)] = random.sample(pairs_test, 2)

        temp = [a, b, c, b, d, e, f, e]
        sentences_test_b.append(temp)
        sentences_number_test_b.append([vocab_map[i] for i in temp])
        x_masked_test_b.append([vocab_map[i] for i in temp])
        y_masked_labels_test_b.append([vocab_map[i] for i in temp][1:])

    x_masked_train = np.array(x_masked_train)
    y_masked_labels_train = np.array(y_masked_labels_train)
    x_masked_test_a = np.array(x_masked_test_a)
    y_masked_labels_test_a = np.array(y_masked_labels_test_a)
    x_masked_test_b = np.array(x_masked_test_b)
    y_masked_labels_test_b = np.array(y_masked_labels_test_b)

    perm = np.random.permutation(len(x_masked_train))
    x_masked_train = x_masked_train[perm]
    y_masked_labels_train = y_masked_labels_train[perm]
    
    num_head = 2

    callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
    inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
    word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
    sinusoidal_embeddings = get_sinusoidal_embeddings(len(x_masked_train[0]), embed_dim)
    encoder_output = word_embeddings + sinusoidal_embeddings

    for i in range(5):
        encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

    encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
    mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
    mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
    adam = Adam()
    mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
    history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                            validation_split = 0.5, callbacks = [callback], 
                            epochs=2000, batch_size=5000, 
                            verbose=0)
    
    acc_a = []
    prob_a = []
    x_test_subset_a = x_masked_test_a[np.random.choice(x_masked_test_a.shape[0], size=1000, replace=False)]

    for sentence_number in x_test_subset_a:
        temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        temp = temp[:,-1,:]
        acc_a.append(1 if temp.argmax() == sentence_number[-1] else 0)
        prob_a.append(temp[0][sentence_number[-1]])
        
    acc_b = []
    prob_b = []
    x_test_subset_b = x_masked_test_b[np.random.choice(x_masked_test_b.shape[0], size=1000, replace=False)]

    for sentence_number in x_test_subset_b:
        temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        temp = temp[:,-1,:]
        acc_b.append(1 if temp.argmax() == sentence_number[-1] else 0)
        prob_b.append(temp[0][sentence_number[-1]])
        
    return ((np.mean(acc_a), np.mean(prob_a)), (np.mean(acc_b), np.mean(prob_b)))


In [10]:
accs_a = 0
probs_a = 0
accs_b = 0
probs_b = 0

for _ in range(10):
    
    ((acc_a, prob_a), (acc_b, prob_b)) = get_accuracy_prob(10)
    
    print((acc_a, prob_a))
    print((acc_b, prob_b))
    
    accs_a += acc_a/10
    probs_a += prob_a/10
    accs_b += acc_b/10
    probs_b += prob_b/10
    
print((accs_a, probs_a))
print((accs_b, probs_b))

(0.425, 0.3226376)
(0.381, 0.30801746)
(0.36, 0.21395385)
(0.392, 0.23340484)
(0.261, 0.19677603)
(0.379, 0.23940429)
(0.398, 0.3181031)
(0.407, 0.31778097)
(0.302, 0.23943798)
(0.361, 0.26378578)
(0.359, 0.293664)
(0.391, 0.31482467)
(0.356, 0.25985482)
(0.383, 0.27994227)
(0.213, 0.16430216)
(0.268, 0.18201005)
(0.427, 0.35032502)
(0.402, 0.34358162)
(0.056, 0.050726455)
(0.063, 0.050630964)
(0.3157, 0.24097810201346875)
(0.3427, 0.25333829298615457)


In [ ]:
accs_a = 0
probs_a = 0
accs_b = 0
probs_b = 0

for _ in range(10):
    
    ((acc_a, prob_a), (acc_b, prob_b)) = get_accuracy_prob(100)
    
    print((acc_a, prob_a))
    print((acc_b, prob_b))
    
    accs_a += acc_a/10
    probs_a += prob_a/10
    accs_b += acc_b/10
    probs_b += prob_b/10
    
print((accs_a, probs_a))
print((accs_b, probs_b))

(0.843, 0.72326577)
(0.837, 0.7803254)
(0.748, 0.6227496)
(0.66, 0.61665004)
(0.896, 0.7346663)
(0.659, 0.6533567)
(0.941, 0.8881785)
(0.978, 0.9491589)
(0.796, 0.6755174)
(0.673, 0.6280614)
(1.0, 0.9975864)
(0.991, 0.98364)
(0.965, 0.8319039)
(0.79, 0.76018786)


In [8]:
for _ in range(3):
    
    ((acc_a, prob_a), (acc_b, prob_b)) = get_accuracy_prob(100)
    
    print((acc_a, prob_a))
    print((acc_b, prob_b))

2024-05-04 20:34:37.840365: W tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-05-04 20:34:37.840401: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-05-04 20:34:37.840419: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gl3390.arc-ts.umich.edu): /proc/driver/nvidia/version does not exist
2024-05-04 20:34:38.033025: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(0.546, 0.51015306)
(0.804, 0.69103795)
(0.598, 0.54464704)
(0.796, 0.7017291)
(1.0, 0.9926709)
(0.997, 0.9845306)


In [9]:
print(np.mean([0.843, 0.748, 0.896, 0.941, 0.796, 1.0, 0.965, 0.546, 0.598, 1.0]))

print(np.mean([0.837, 0.66, 0.659, 0.978, 0.673, 0.991, 0.79, 0.804, 0.796, 0.997]))

0.8333
0.8185
